In [1]:
# load data and shuffle it
import glob
import cv2
import numpy as np
import pandas as pd
"""
img = cv2.imread(r'data\TrainingData\0003.jpg')
cv2.imshow('0003 test', img)
img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
cv2.imshow('after downsampling', img)
cv2.waitKey()
"""
def bin2class(row):
    if row[0] == False and row[1] == False: return 0
    elif row[0] == False and row[1] == True: return 1
    elif row[0] == True and row[1] == False: return 2
    elif row[0] == True and row[1] == True: return 3


images = [cv2.imread(file) for file in glob.glob(r'data\TrainingData\*.jpg')]
#rsize = 152
#images = [cv2.resize(cv2.imread(file),(rsize,rsize),interpolation=cv2.INTER_AREA) for file in glob.glob(r'data\TrainingData\*.jpg')]
x = np.array(images, dtype=np.float64)          # x.shape = N, height, width, channel

tags = pd.read_excel(r'data\DataInfo.xlsx', true_values=["'High'", "'MIBC'"], false_values=["'Low'","'NMIBC'"])
tags = tags.iloc[:, 1:].to_numpy(dtype=int)     # tags.shape = N, grading, staging
y = np.apply_along_axis(bin2class, 1, tags)     # y.shape = (N, ) one-hot encoding

# shuffle data
N = y.shape[0]
shuffle_mask = np.random.shuffle(np.arange(N))
x[:] = x[shuffle_mask]
y[:] = y[shuffle_mask]

print(x.shape)
print(y.shape)
num_classes = 4

(478, 512, 512, 3)
(478,)


In [2]:
num_classes = 4
"""
======locally connected layers======
    (F, C, H, W)    S next, params
L1: (16, 3, 11, 11) 3 40,   9318400
L2: (16, 16, 9, 9)  1 32,   21250048
L3: (16, 16, 9, 9)  1 24,   11953152
======fully connected layers========
F1: (24*24*16, 300)
F2: (300, 300)
F3: (300, 300)
======output layer==================
F:  (300, 4)
softmax
"""

'\n======locally connected layers======\n    (F, C, H, W)    S next, params\nL1: (16, 3, 11, 11) 3 40,   9318400\nL2: (16, 16, 9, 9)  1 32,   21250048\nL3: (16, 16, 9, 9)  1 24,   11953152\n======fully connected layers========\nF1: (24*24*16, 300)\nF2: (300, 300)\nF3: (300, 300)\n======output layer==================\nF:  (300, 4)\nsoftmax\n'

In [3]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import LocallyConnected2D, BatchNormalization, Conv2D, MaxPooling2D
from keras.layers import LeakyReLU, Dense, Dropout, Flatten, Softmax
from keras import regularizers
from keras.utils import to_categorical

def establish_model():
    model = Sequential()
    # N, 3, 512, 512, preprocess
    model.add(Conv2D(32, 3, padding='same', input_shape=(512, 512, 3), kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization(axis=1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    # N, 32, 256, 256
    model.add(Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization(axis=1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D())
    
    # N, 3 ,128, 128
    model.add(LocallyConnected2D(16, 11, strides=(3, 3), kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization(axis=1))
    model.add(LeakyReLU(alpha=0.2))
    #model.add(Dropout(0.5))
    # N, 16, 40, 40
    model.add(LocallyConnected2D(16, 9, kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization(axis=1))
    model.add(LeakyReLU(alpha=0.2))
    #model.add(Dropout(0.5))
    # N, 16, 32, 32
    model.add(LocallyConnected2D(16, 9, kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization(axis=1))
    model.add(LeakyReLU(alpha=0.2))
    #model.add(Dropout(0.5))
    # N, 16, 24, 24
    model.add(Flatten())
    # N, 16*24*24
    model.add(Dense(300, kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization(axis=1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))

    model.add(Dense(300, kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization(axis=1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(Softmax())
    
    return model

Using TensorFlow backend.


In [ ]:
# overfitting on small set of data

x_pre, y_pre = x[:30], y[:30]
y_pre = to_categorical(y_pre, num_classes)

model = establish_model()
optim = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
hist = model.fit(x_pre, y_pre, batch_size=15, epochs=20, shuffle=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20


In [ ]:
import matplotlib.pyplot as plt

plt.subplot(2,1,1)
plt.plot(hist.history['acc'])
#plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.subplot(2,1,2)
plt.plot(hist.history['loss'])
#plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# split for cross validation
delim = [120, 240, 360]
xs = np.split(x, delim)
ys = np.split(y, delim)
num_classes = 4

# validation
loss, acc = 0, 0
for vali in range(num_classes):
    print("====== Cross validation step %d ======"%(vali+1))
    x_train = np.vstack([xs[i] for i in range(num_classes) if i != vali])
    x_val = xs[vali]
    y_train = to_categorical(np.hstack([ys[i] for i in range(num_classes) if i != vali]), num_classes)
    y_val = to_categorical(ys[vali], num_classes)
    
    # establish model
    model = establish_model()
    
    optim = optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    hist = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=16, epochs=20, shuffle=True)
    
    plt.figure()
    plt.subplot(1,2,1)
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.subplot(1,2,2)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
    loss += hist.history['val_acc'][-1]
    acc += hist.history['val_loss'][-1]

print("average loss: %f"%(loss/num_classes))
print("average accuracy: %f"(%acc/num_classes))